In [12]:
import pickle
import tensorflow_hub as hub
from tqdm import tqdm
import tensorflow_text
import numpy as np
import tensorflow as tf
import pandas as pd

file = open("models/model_3.pickle", "rb")
model = pickle.load(file)
file.close()

use = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

Keras model archive loading:
File Name                                             Modified             Size
variables.h5                                   2022-12-20 00:41:40      2001384
config.json                                    2022-12-20 00:41:40         2552
metadata.json                                  2022-12-20 00:41:40           64
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dense_2
.........vars
............0
............1
......dropout
.........vars
......dropout_1
.........vars
...metrics
......mean
.........vars
............0
............1
......mean_metric_wrapper
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars


In [15]:
def evaluate_multiple(strings: list):
    data_to_test = np.array(strings)

    i_want_this_tested = []
    for r in data_to_test:
        emb = use(r)
        review_emb = tf.reshape(emb, [-1]).numpy()
        i_want_this_tested.append(review_emb)
    i_want_this_tested = np.array(i_want_this_tested)

    return list(model.predict(i_want_this_tested))


def evaluate_single(single: str):
    return evaluate_multiple([single])


desired_strings = ["Twitter is horrible.", "I love twitter!", "I dunno :(", "bahaha", "Starlink now has more than 1,000,000 active subscribers – thank you to all customers and members of the Starlink team who contributed to this", "Those who want power are the ones who least deserve it", "Should I step down from twitter?"]

evaluated = evaluate_multiple(desired_strings)

output = {}
for i in range(len(desired_strings)):
    output[desired_strings[i]] = "POSITIVE" if evaluated[i].argmax() == 1 else "NEGATIVE"

output

1/1 [==============================] - 0s 488ms/step


{'Twitter is horrible.': 'NEGATIVE',
 'I love twitter!': 'POSITIVE',
 'I dunno :(': 'NEGATIVE',
 'bahaha': 'POSITIVE',
 'Starlink now has more than 1,000,000 active subscribers – thank you to all customers and members of the Starlink team who contributed to this': 'POSITIVE',
 'Those who want power are the ones who least deserve it': 'POSITIVE',
 'Should I step down from twitter?': 'NEGATIVE'}

In [9]:
pd.DataFrame(
    {
        "tweet": ["hello", "i despise you"],
        "date": ["1,2,3", "4,5,6"],
        "user": ["deez", "nas"]
    }
)

NameError: name 'pd' is not defined